## Acesso a base globalstore (pública) no formato csv.

In [ ]:
globalstore = pd.read_csv('globalstore.csv')
globalstore

In [ ]:
globalstore.info()

In [ ]:
globalstore = globalstore.rename(
    columns = {
        'Row ID': 'items_sequence',
        'Order ID': 'order_number',
        'Order Date': 'order_date',
        'Ship Date': 'ship_date',
        'Ship Mode': 'ship_mode',
        'Customer ID': 'customer_id',
        'Customer Name': 'customer_name',
        'Segment': 'segment',
        'State': 'state',
        'City': 'city',
        'Country': 'country',
        'Postal Code': 'postal_code',
        'Region': 'region',
        'Market': 'market',
        'Product ID': 'product_id',
        'Product Name': 'product_name',
        'Category': 'category',
        'Sub-Category': 'subcategory',
        'Sales': 'sales',
        'Quantity': 'quantity',
        'Profit': 'profit',
        'Discount': 'discount',
        'Shipping Cost': 'shipping_cost',
        'Order Priority': 'order_priority',
    }
)

In [ ]:
globalstore

In [ ]:
globalstore.info()

In [ ]:
globalstore['Order Date'] = globalstore['Order Date'].astype('datetime64[ns]')
globalstore['Ship Date'] = globalstore['Ship Date'].astype('datetime64[ns]')
globalstore.info()

## Pacotes de Acesso:
- Pandas: acesso e maipulação de dados
- sqlite3: conexao com banco relacional sqlite

In [ ]:
import sqlite3 as sql

## String de conexão 
O banco de dados sqlite gera um instancia local. Quando for acesso a um servidor passar a url. Alguns bancos de dados requerem mais parâmetros para realizar a conexão.


In [ ]:
conexao = sql.connect('globalstore.db')

## Função de normalização - Aplicação da terceira forma normal (dependencia transitiva)

A dependência transitiva ocorre quando um ou mais atributos estão diretamente relacionados ao conteúdo de um atributo, não pertencente a chave primária (identificador) da tabela

Os parâmetros informados são:
dataframe: conjunto de dados originais;
colunas: lista de colunas 

In [ ]:
def normalizacao(dataframe, colunas, apelido):
    if len(colunas) == 1:
        normalizado = [[s, d] for s, d in enumerate(dataframe[colunas[0]].unique())]
        normalizado = pd.DataFrame(
            normalizado,
            columns = [
                apelido+'_sequence',
                'description'
            ]
        )
        dataframe = dataframe.merge(
            normalizado,
            left_on = colunas,
            right_on = 'description',
            how = 'left'
        )
        dataframe = dataframe.drop(
            columns = [colunas, 'description']
        )
    else:
        normalizado = dataframe[colunas].drop_duplicates().reset_index()
        normalizado = normalizado.rename(
            columns={'index': apelido+'_sequence'}
        )
        dataframe = dataframe.merge(
            normalizado, 
            right_on=colunas,
            left_on=colunas, 
            how='left'
        )
        for coluna in dataframe.columns:
            if coluna.endswith('_x') or coluna.endswith('_y') or coluna in colunas:
                dataframe = dataframe.drop(columns=coluna)        
    return dataframe, normalizado

In [ ]:
items, orders = normalizacao(
    globalstore, 
    [
        'order_number',
        'order_date',
        'ship_date',
        'ship_mode',
        'customer_id',
        'customer_name',
        'segment',
        'city',
        'state',
        'country',
        'postal_code',
        'market',
        'region',
        'order_priority'
    ], 
    'ord', 
)

In [ ]:
orders

,ord_sequence,order_number,order_date,ship_date,ship_mode,customer_id,customer_name,segment,city,state,country,postal_code,market,region,order_priority
0,0,MX-2014-143658,10-02-14,10-06-14,Standard Class,SC-20575,Sonia Cooley,Consumer,Mexico City,Distrito Federal,Mexico,NaN,LATAM,North,Medium
1,1,MX-2012-155047,10-15-12,10-20-12,Standard Class,KW-16570,Kelly Williams,Consumer,Dos Quebradas,Risaralda,Colombia,NaN,LATAM,South,Medium
2,6,MX-2013-134096,09-27-13,10-01-13,Standard Class,DP-13000,Darren Powers,Consumer,São Paulo,São Paulo,Brazil,NaN,LATAM,South,Medium
3,11,MX-2013-156335,03-05-13,03-12-13,Standard Class,TB-21250,Tim Brockman,Consumer,Managua,Managua,Nicaragua,NaN,LATAM,Central,Medium
4,13,MX-2014-121923,09-24-14,10-01-14,Standard Class,PK-18910,Paul Knutson,Home Office,Managua,Managua,Nicaragua,NaN,LATAM,Central,Low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25751,51276,TU-2013-7800,08-09-13,08-11-13,Second Class,MY-8295,Muhammed Yedwab,Corporate,Adana,Adana,Turkey,NaN,EMEA,EMEA,Medium
25752,51280,IV-2012-7600,09-21-12,09-24-12,Second Class,TC-11145,Theresa Coyne,Corporate,Bouake,Vallee Du Bandama,Cote d'Ivoire,NaN,Africa,Africa,Medium
25753,51281,RS-2014-4620,08-14-14,08-16-14,Second Class,CR-2625,Corey Roper,Home Office,Astrakhan',Astrakhan',Russia,NaN,EMEA,EMEA,Medium
25754,51284,HU-2012-7730,09-05-12,09-07-12,First Class,NF-8385,Natalie Fritzler,Consumer,Budapest,Budapest,Hungary,NaN,EMEA,EMEA,High


In [ ]:
items

,items_sequence,product_id,category,subcategory,product_name,sales,quantity,discount,profit,shipping_cost,ord_sequence
0,1,OFF-LA-10002782,Office Supplies,Labels,"Hon File Folder Labels, Adjustable",13.08,3,0.0,4.56,1.033,0
1,2,FUR-FU-10004015,Furniture,Furnishings,"Tenex Clock, Durable",252.16,8,0.0,90.72,13.449,1
2,3,FUR-BO-10002352,Furniture,Bookcases,"Ikea 3-Shelf Cabinet, Mobile",193.28,2,0.0,54.08,9.627,1
3,4,OFF-BI-10004428,Office Supplies,Binders,"Cardinal Binder, Clear",35.44,4,0.0,4.96,1.371,1
4,5,OFF-AR-10004594,Office Supplies,Art,"Sanford Canvas, Water Color",71.60,2,0.0,11.44,3.787,1
...,...,...,...,...,...,...,...,...,...,...,...
51285,51286,OFF-AVE-10004570,Office Supplies,Binders,"Avery Binder Covers, Economy",11.07,1,0.0,3.42,1.980,51284
51286,51287,TEC-LOG-10004419,Technology,Accessories,"Logitech Flash Drive, USB",61.44,2,0.0,18.42,13.020,51284
51287,51288,OFF-BOS-10002705,Office Supplies,Art,"Boston Highlighters, Fluorescent",80.52,4,0.0,20.88,8.780,51284
51288,51289,OFF-ENE-10004132,Office Supplies,Paper,"Enermax Computer Printout Paper, 8.5 x 11",130.44,4,0.0,33.84,18.970,51284


Exercício:
- Normalizar a tabela items com product;
- Normalizar a tablea product com subcategory;
- Normalizar a tabela subcategory com category;
- Normalizar a tabela orders para client;
- Normalizar a tabela cliente para city;
- Normalizar a tabela city para market;


## Dados em conjuntos para uso durante o processo
recomenda-se formato parquet ou feather.

In [ ]:
orders.to_parquet('orders.parquet')

In [ ]:
orders.read_parquet('orders.parquet')
orders

Atividade: Salvar todas as tabelas no formato parquet

Atividade Extra: 
- remontar a estrutura original com a função merge
- criar um banco de dados com os dados referentes aos mercados 'LATAM' e 'US'  

## Dados para banco de dados relacional


In [ ]:
orders.to_sql(
    name='orders',
    con=conexao,
    if_exists='replace'
)

In [ ]:
step = 10000
for linhas in range(0, len(items), step):
    print(linhas)
    items[linhas:linhas+step].to_sql(
        name='items',
        con=conexao,
        if_exists='append'       
    )

## items.to_sql(name='outros_items', con=conexao, if_exists='append', chunksize=10000)    

In [ ]:
conexao_normalizada = sql.connect('globalstore_normalizada.db')

In [ ]:
for tabela in ['category', 'subcategory', 'products', 'orders', 'items', 'customer', 'city', 'region']:
    eval(tabela).to_sql(
        tabela,
        con = conexao_normalizada,
        if_exists = 'replace'
    )

Atividade: Salvar as demais tabela no banco de dados relacional 

In [ ]:
with conexao:
    data = conexao.execute("select * from items")
    for row in data:
        print(row)

Atividade: 
- Listar os clientes que compraram os produtos da categoria 'Furniture';
- Listar a quantidade de produtos agrupados por pais e cidade;